# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** Givandra Haikal Adjie
- **Email:** givhaikal@gmail.com
- **ID Dicoding:** givandra_haikal_Il2s

## Menentukan Pertanyaan Bisnis

- Pada masa atau rentang waktu apakah rental sepeda sangat laku?
- Bagaimana pengaruh cuaca (suhu, angin, dan kelembaban) terhadap banyaknya rental sepeda?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import plotly.express as px
from IPython.display import Markdown 
import datetime as dt

## Data Wrangling

### Gathering Data

Dataset: bike sharing

The dataset contains 2 files:
- hour.csv : bike sharing counts aggregated on hourly basis. Records: 17379 hours
- day.csv : bike sharing counts aggregated on daily basis. Records: 731 days

From the readme file, both hour.csv and day.csv have the following fields, except hr which is not available in day.csv
	
- `instant`: record index
- `dteday` : date
- `season` : season (1:spring, 2:summer, 3:fall, 4:winter)
- `yr` : year (0: 2011, 1:2012)
- `mnth` : month ( 1 to 12)
- `hr` : hour (0 to 23)
- `holiday` : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- `weekday` : day of the week
- `workingday` : if day is neither weekend nor holiday is 1, otherwise is 0.
- ` weathersit` : 
	- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
	- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
	- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
	- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- `temp` : Normalized temperature in Celsius. The values are divided to 41 (max)
- `atemp`: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- `hum`: Normalized humidity. The values are divided to 100 (max)
- `windspeed`: Normalized wind speed. The values are divided to 67 (max)
- `casual`: count of casual users
- `registered`: count of registered users
- `cnt`: count of total rental bikes including both casual and registered

In [ ]:
# - hour.csv : bike sharing counts aggregated on hourly basis. Records: 17379 hours
# - day.csv : bike sharing counts aggregated on daily basis. Records: 731 days

df_day  = pd.read_csv('dataset/day.csv')
#df_hour = pd.read_csv('dataset/hour.csv')

display(Markdown("## Day"))
display(df_day)

# display(Markdown("## Hour"))
# display(df_hour)

Dari hasil load dataset ini, jumlah row dan kolom kolomnya sudah sesuai dengan yang tertulis di Readme.md. Saya akan melanjutkan analisis untuk dataset daily agar lebih simpel

### Assessing Data

Mengecek informasi dataframe

In [ ]:
df_day.info()

Kita menemukan dataset tidak memiliki nilai kosong untuk semua kolom yang ada

In [ ]:
df_day.duplicated().sum()

Dataset juga tidak memiliki row yang duplikat

In [ ]:
df_day.head()

Mengecek outliers

In [ ]:
Q1 = df_day['cnt'].quantile(0.25)
Q3 = df_day['cnt'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify the outliers
outliers = df_day[(df_day['cnt'] < lower_bound) | (df_day['cnt'] > upper_bound)]
outliers

In [ ]:
print(f"{lower_bound= }")
print(f"{upper_bound= }")
print("Minimal", df_day['cnt'].min())
print("Maximal", df_day['cnt'].max())

Dataset juga tidak mempunyai outlier, semua data masuk kedalam IQR

### Cleaning Data

Dataset sudah cukup bersih untuk dilakukan analisis lebih lanjut. Saya akan mengubah nilai data saja dari bentuk angka menjadi bentuk stringnya

In [ ]:
df_day = df_day.replace({
    'season' :{ 1: 'Spring', 2: 'Summer', 3: 'Fall', 4: 'Winter'},
    'weekday':{0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'},
    'yr' : {0: 2011, 1: 2012 }
})

In [ ]:
df_day.head()

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
df_day.describe()

In [ ]:
season_data = df_day.groupby(['season', 'yr']).agg({
    'cnt': 'sum'
}).reset_index(level=[0,1])

# season_data.columns
px.histogram(season_data, x='season', y='cnt', color='yr', barmode='group', 
             title="Banyak rental sepeda per musim")

In [ ]:
weather_data = df_day.groupby(['season', 'yr', 'weathersit']).count().reset_index(level=[0,1,2])

# weather_data
px.histogram(weather_data, x='season', y='cnt', color='weathersit', barmode='group', 
             title="Kondisi cuaca per musim").update_layout(
             xaxis_title="Season", 
             yaxis_title="Count",
)

In [ ]:
week_data = df_day.groupby(['weekday']).agg({
    'cnt': 'sum'
}).reset_index()

px.bar(week_data, x='weekday', y='cnt' )

In [ ]:
px.scatter(df_day, x='temp', y='cnt', )

In [ ]:
px.scatter(df_day, x='windspeed', y='cnt', )


In [ ]:
px.scatter(df_day, x='hum', y='cnt', )

## Visualization & Explanatory Analysis

### Pertanyaan 1:
Pada masa atau rentang waktu apakah rental sepeda sangat laku?

In [ ]:
px.histogram(season_data, x='season', y='cnt', color='yr', barmode='group', 
             title="Banyak rental sepeda per musim")

Dari explorasi di atas, temuan menarik ditemukan rental sepeda paling laku dimusim gugur. Ini dimungkinkan dikarenakan suhu yang nyaman untuk bersepeda di luar, tidak terlalu panas dan tidak terlalu dingin. Kita juga menemmukan musim semi sebagai musim yang paling sepi aktivitas rental sepeda. Ini mungkin dikarenakan curah hujan yang cukup tinggi. 

Dibawah ini merupakan plot untuk keadaan cuaca sepanjang musim

In [ ]:
px.histogram(weather_data, x='season', y='cnt', color='weathersit', barmode='group', 
             title="Kondisi cuaca per musim").update_layout(
             xaxis_title="Season", 
             yaxis_title="Count",
)

Kita melihat musim gugur sebagai musim dengan kondisi cuaca yang paling ideal untuk bersepeda dimana banyak hari tidak hujan atau mendung. Jika kita perhatikan musim semi dengan musim panas dan musim dingin, kita tidak melihat begitu perbedaan. Kemungkinan ada alasan lain mengapa musim semi memiliki minat bersepeda yang lebih rendah dari musim lainnya.

### Pertanyaan 2:
Bagaimana pengaruh cuaca (suhu, angin, dan kelembaban) terhadap banyaknya rental sepeda?

In [ ]:
display(px.scatter(df_day, x='temp', y='cnt', title='Temperatur vs Jumlah rental sepeda').update_layout(
             xaxis_title="Temperatur", 
             yaxis_title="Jumlah",
))
display(px.scatter(df_day, x='windspeed', y='cnt', title='Kecepatan angin vs Jumlah rental sepeda').update_layout(
             xaxis_title="Kecepatan angin", 
             yaxis_title="Jumlah"
))
display(px.scatter(df_day, x='hum', y='cnt', title='Kelembaban vs Jumlah rental sepeda').update_layout(
             xaxis_title="Kelembaban", 
             yaxis_title="Jumlah"
))

Dari ketiga parameter yang diduga memiliki pengaruh ke jumlah rental sepeda, kita menemukan hanya temperatur yang memiliki korelasi kuat dengan jumlah rental sepeda. Dua parameter lain, kecepatan angin dan kelembaban tidak begitu terlihat korelasinya dan terlihat random. Untuk temperatur, parameter ini berkorelasi positif dengan jumlah rental sepeda. Namun, jika dilihat lebih lanjut, ada titik yang menjadi titik putar balik dimana lebih dari suhu ini akan menjadi terlalu panas dan rental sepeda menjadi berkurang. 

## Conclusion

- Kita menemukan bahwa musim gugur adalah musim dengan minat bersepeda paling tinggi dan musim semi merupakan musim yang kurang diminati untuk bersepeda
- Parameter cuaca yang paling berpengaruh ke jumlah rental sepeda adalah suhu. Parameter lain seperti angin dan kelembaban memiliki korelasi yang lemah. Suhu yang hangat lebih diminati untuk menjadi hari untuk bersepeda.